In [ ]:
# %pip install -r requirements.txt

In [ ]:
# !huggingface-cli download lmsys/vicuna-7b-v1.3 --local-dir '/root/SD-research/base_models/vicuna'
# !huggingface-cli download yuhuili/EAGLE-Vicuna-7B-v1.3 --local-dir '/root/SD-research/ea_models/vicuna'

In [23]:
!huggingface-cli download meta-llama/Llama-2-7b-chat-hf --local-dir '/root/SD-research/base_models/llama2'
!huggingface-cli download yuhuili/EAGLE-llama2-chat-7B --local-dir '/root/SD-research/ea_models/llama2'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fetching 16 files: 100%|██████████████████████| 16/16 [00:00<00:00, 5904.87it/s]
/root/SD-research/base_models/llama2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fetching 4 files:   0%|                                   | 0/4 [00:00<?, ?it/s]Downloading 'README.md' to '/root/SD-research/ea_models/llama2/.cache/huggingface/download/README.md.154df8298fab5ecf322016157858e08cd1bccbe1.incomplete'

README.md: 100%|██████████████████████████████| 28.0/28.0 [00:00<00:00, 263kB/s]
Download complete. Moving file to /root/SD-research/ea_models/llama2/README.md

.gitattributes: 100%|██████████████████████| 1.52k/1.52k [00:00<00:00, 22.7MB/s]
Download complete. Moving file to /root/SD-research/ea_models/llama2/.gitattributes
Fetching 4 files:  25%|██████▊                    | 1/4 [00:00<00:01,  1.97it/s]Downloading 'pytorch_model.bin' to '/root/SD-research/ea_models/llama2/.cache/huggingface/download/pytorch_model.bin.3f1e4f6bef7befeba9d08754f98088799d754250b20fc61a46081e8ea0ca978b.incomplete'

config.json: 100%|█████████████████████████████| 560/560 [00:00<00:00, 4.48MB/s]
Download complete. Moving file to /root/SD-research/ea_models/llama2/config.json
Fe

In [6]:
import torch
import argparse
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import json
import csv
from pathlib import Path
import logging
from datetime import datetime
from typing import List, Dict
import yaml
import os
from eagle.model.ea_model import EaModel
from fastchat.model import get_conversation_template

In [ ]:
model = EaModel.from_pretrained(
    base_model_path='/root/SD-research/base_models/vicuna',
    ea_model_path='/root/SD-research/ea_models/vicuna',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)
model.eval() 

In [ ]:
for i in tqdm(range(4)):
    prompts = []
    # load input data
    segment_string = f'{500*i}~{500*(i+1)}'
    input_filename = f"../data/ArXiv/4K/ArXiv_4K_{segment_string}.json"
    with open(input_filename, 'r', encoding='utf-8') as f:
        raw_input = json.load(f)
        input_data = [item['final_input'] for item in raw_input]
        
    for item in input_data :
        conv = get_conversation_template("vicuna")
        conv.append_message(conv.roles[0], item)
        conv.append_message(conv.roles[1], None)
        prompts.append(conv.get_prompt())
        
    start_time = datetime.now()
    input_s=model.tokenizer(prompts,return_tensors="pt",padding=True).to("cuda")
    output_ids=model.eagenerate(input_s.input_ids,input_s.attention_mask,temperature=0.0,max_new_tokens=512,top_k=15)
    output=model.tokenizer.batch_decode(output_ids)    
    inference_time = (datetime.now() - start_time).total_seconds()
    print(inference_time)
    
    start_time_v = datetime.now()    
    output_ids, new_token, idx=model.naive_generate(input_s.input_ids,input_s.attention_mask,temperature=0.0,max_new_tokens=512,top_k=15,log=True)
    inference_time_v = (datetime.now() - start_time_v).total_seconds()
    print(inference_time_v)

  0%|          | 0/4 [00:00<?, ?it/s]


TypeError: EaModel.naive_generate() got multiple values for argument 'temperature'

In [39]:
i = 1
segment_string = f'{500*i}~{500*(i+1)}'
input_filename = f"../data/ArXiv/4K/ArXiv_4K_{segment_string}.json"
with open(input_filename, 'r', encoding='utf-8') as f:
    raw_input = json.load(f)
    input_data = [item['final_input'] for item in raw_input]        
    print(input_data[0])

conv = get_conversation_template("vicuna")
conv.append_message(conv.roles[0], input_data[0])
conv.append_message(conv.roles[1],None)
prompt = conv.get_prompt()
prompt

You will receive the full text of a scientific paper. Your task is to write a formal abstract (under 300 words) that includes:

- The primary objective or research question.
- A brief description of the methodology.
- The main findings or results.
- The significance or implications of the findings.

Use clear, professional, and concise language suitable for publication.

### Paper content

there are several reasons to believe that a population of intergalactic globular clusters ( igcs ) should exist outside of galaxies :    \(1 ) the jeans mass at recombination was @xmath0 solar masses , and hence globular cluster sized objects could have formed wherever the local density of matter was high enough . \(2 ) many galaxies may have met their demise over a hubble time as a result of collisions and tidal disruption . globular clusters are likely to survive the disruption of their parent galaxy , resulting in the gradual accumulation of a population of igcs . intergalactic stars , planetary n

"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: You will receive the full text of a scientific paper. Your task is to write a formal abstract (under 300 words) that includes:\n\n- The primary objective or research question.\n- A brief description of the methodology.\n- The main findings or results.\n- The significance or implications of the findings.\n\nUse clear, professional, and concise language suitable for publication.\n\n### Paper content\n\nthere are several reasons to believe that a population of intergalactic globular clusters ( igcs ) should exist outside of galaxies :    \\(1 ) the jeans mass at recombination was @xmath0 solar masses , and hence globular cluster sized objects could have formed wherever the local density of matter was high enough . \\(2 ) many galaxies may have met their demise over a hubble time as a result of collisions and tidal disruption . gl

In [ ]:


input_ids=model.tokenizer([prompt]).input_ids
input_ids = torch.as_tensor(input_ids).cuda()
output_ids=model.eagenerate(input_ids,temperature=0.5,max_new_tokens=512)
output=model.tokenizer.decode(output_ids[0])

In [41]:
output_ids

tensor([[    1,   319, 13563,  ...,  6757, 29889,     2]], device='cuda:0')

In [42]:
prompt

"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: You will receive the full text of a scientific paper. Your task is to write a formal abstract (under 300 words) that includes:\n\n- The primary objective or research question.\n- A brief description of the methodology.\n- The main findings or results.\n- The significance or implications of the findings.\n\nUse clear, professional, and concise language suitable for publication.\n\n### Paper content\n\nthere are several reasons to believe that a population of intergalactic globular clusters ( igcs ) should exist outside of galaxies :    \\(1 ) the jeans mass at recombination was @xmath0 solar masses , and hence globular cluster sized objects could have formed wherever the local density of matter was high enough . \\(2 ) many galaxies may have met their demise over a hubble time as a result of collisions and tidal disruption . gl

In [47]:
import re
re.split('ASSISTANT: ',output)[-1]


'The primary objective of this study was to investigate the existence of intergalactic globular clusters (IGCs) outside of galaxies. The research question was whether IGCs exist and could be detected in the field of the rich galaxy cluster A1185. The methodology involved acquiring deep, multicolored images of the A1185 field using the Hubble Space Telescope (HST) and the new Advanced Camera for Surveys (ACS). The authors aimed to detect the peak of the assumed universal Gaussian-like globular cluster luminosity function and use color information to infer the metallicities of the IGCs. The main findings were that the authors were able to reach sufficiently faint magnitudes to reliably detect the luminosity function turnover. Preliminary analysis of the data revealed that the number and colors of the IGCs will provide constraints on the number and types of galaxies that have been destroyed or stripped over a Hubble time. The significance of the findings is that the existence of IGCs migh